# Invoke SageMaker Autopilot Model from Athena

### Install PyAthena

In [ ]:
!pip install -q PyAthena==1.8.0

In [ ]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
from pyathena.util import as_pandas

In [ ]:
import boto3
import sagemaker
import pandas as pd

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role() 

# Create an Athena Table with Sample Reviews

In [ ]:
# Set S3 prefixes
tsv_prefix = 'amazon-reviews-pds/tsv'

# Set Athena parameters
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'
table_name = 'product_reviews'

In [ ]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = 's3://{}/athena/staging'.format(bucket)

In [ ]:
# SQL statement to execute
statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS 
SELECT review_id, review_body 
FROM {}.{}
""".format(database_name, table_name, database_name, table_name_tsv)

print(statement)

In [ ]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [ ]:
statement = 'SELECT * FROM {}.{} LIMIT 10'.format(database_name, table_name)
cursor.execute(statement)

In [ ]:
df_show = as_pandas(cursor)
df_show

# Retrieve Autopilot Endpoint Name

In [ ]:
%store -r autopilot_endpoint_name

In [ ]:
print(autopilot_endpoint_name)

In [ ]:
statement = """
USING FUNCTION predict_star_rating(review_body VARCHAR) 
    RETURNS VARCHAR TYPE
    SAGEMAKER_INVOKE_ENDPOINT WITH (sagemaker_endpoint = '{}'
)
SELECT review_id, review_body, predict_star_rating(REPLACE(review_body, ',', ' ')) AS star_rating 
    FROM {}.{} LIMIT 10
    """.format(autopilot_endpoint_name, database_name, table_name)

print(statement)

# Add the Required `AmazonAthenaPreviewFunctionality` Work Group to Use This Preview Feature

In [ ]:
import boto3

client = boto3.client('athena')
response = client.create_work_group(Name='AmazonAthenaPreviewFunctionality')                                    

print(response)

# Query the Autopilot Endpoint using Data from the Athena Table

In [ ]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, 
                 s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement, 
               work_group='AmazonAthenaPreviewFunctionality')

In [ ]:
df = as_pandas(cursor)
df.head(5)